In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


In [ ]:
df_tr = pd.read_csv('../input/intras/train_data.csv')
df_ts = pd.read_csv('../input/intrasb/test_data.csv')
print("Train:",df_tr.shape)
print("Test:",df_ts.shape)


In [ ]:
print('Train Dataset Infomarion')
print ("Rows     : " ,df_tr.shape[0])
print ("Columns  : " ,df_tr.shape[1])
print ("\nFeatures : \n" ,df_tr.columns.tolist())
print ("\nMissing values :  ",df_tr.isnull().sum().values.sum())
print ("\nUnique values :  \n",df_tr.nunique())

In [ ]:

sns.heatmap(df_tr.isnull())

In [ ]:
df_tr_fil = df_tr[df_tr['null count'] <= 0]
print(df_tr_fil.shape)
print(df_ts.shape)

In [ ]:
sns.heatmap(df_tr_fil.isnull())

In [ ]:
sns.heatmap(df_ts.isnull())



In [ ]:
import missingno as msno
#Filling missing data with mean/median
df_ts["MasVnrArea"] = df_ts["MasVnrArea"].replace(np.NaN, df_ts["MasVnrArea"].mean())
df_ts["GarageYrBlt"] = df_ts["GarageYrBlt"].replace(np.NaN, df_ts["GarageYrBlt"].mean())
#df_ts["LotFrontage"] = df_ts["LotFrontage"].replace(np.NaN, df_ts["LotFrontage"].median())
#Filling missing categorical data with most frequent value

df_ts["BsmtFinType1"] = df_ts["BsmtFinType1"].fillna('U')
df_ts["BsmtFinType2"] = df_ts["BsmtFinType2"].fillna('U')
df_ts["GarageType"] = df_ts["GarageType"].fillna('U')

cateogry_columns=df_ts.select_dtypes(include=['object']).columns.tolist()
integer_columns=df_ts.select_dtypes(include=['int64','float64']).columns.tolist()

for column in df_ts:
    if df_ts[column].isnull().any():
        if(column in cateogry_columns):
            df_ts[column]=df_ts[column].fillna(df_ts[column].mode()[0])
        else:
            df_ts[column]=df_ts[column].fillna(df_ts[column].median)

msno.matrix(df_ts)

In [ ]:
cols_tr = df_tr_fil.columns
num_cols_tr= df_tr_fil._get_numeric_data().columns
cat_cols_tr = list(set(cols_tr) - set(num_cols_tr))
print(cat_cols_tr)
cols = df_ts.columns
num_cols = df_ts._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

In [ ]:
import sklearn
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None  # default='warn'
  
df_tr_fil[cat_cols_tr] = df_tr_fil[cat_cols_tr].apply(LabelEncoder().fit_transform)
label_encoder=LabelEncoder()
df_ts[cat_cols] = df_ts[cat_cols].apply(LabelEncoder().fit_transform)


In [ ]:
df_tr_fil.head()

In [ ]:
df_tr_fil.drop('Unnamed: 0',axis=1)
#df_ts_fil.drop('Unnamed: 0',axis=1)

In [ ]:
df_tr_fil.drop('null count',axis=1)
y=df_tr_fil['SalePrice']
X=df_tr_fil.drop(['SalePrice','Unnamed: 0'],axis=1)
Y=df_ts.drop('Unnamed: 0',axis=1)
Y.shape

In [ ]:
# Splitting the dataset into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.66, random_state = 0)

In [ ]:
X_train.head()

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.5603, gamma=0.0368, 
                             eta=0.01, max_depth=6, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =42)

In [ ]:
model_xgb.fit(X_train,y_train)

In [ ]:
y_pred = model_xgb.predict(X_val)

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(y_val, y_pred))
print('all gives RMSE is:',rms)

In [ ]:
Y_pred=model_xgb.predict(Y)
label_encoder.fit(Y_pred)
#pred_labels = label_encoder.inverse_transform(Y_pred)
pred_final = pd.DataFrame(Y_pred)
#pred_final.insert(0, "Id", id_col, True)
pred_final.describe(include='all')

In [ ]:
pred_final.to_csv('Fin.csv', index=False)

In [ ]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
NN_model = Sequential()
keras.regularizers.l2(0.02)
keras.regularizers.l1_l2(l1=0.02, l2=0.02)

# The Input Layer :
NN_model.add(Dense(75, kernel_initializer='normal',input_dim = X_train.shape[1], activation='linear'))

# The Hidden Layers :
NN_model.add(Dense(100, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(66, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(44, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(X_train, y_train, epochs=300, batch_size=19, validation_split = 0.1, callbacks=callbacks_list)

In [ ]:
wights_file = 'Weights-298--18231.20117.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


In [ ]:
preds=NN_model.predict(X_val)

rms2 = np.sqrt(mean_squared_error(y_val, preds))
print('all gives RMSE is:',rms2)


In [ ]:
#label_encoder.fit(preds)
#pred_labels = label_encoder.inverse_transform(Y_pred)
preds=NN_model.predict(Y)
preds2 = pd.DataFrame(preds)
#pred_final.insert(0, "Id", id_col, True)
preds2.describe(include='all')
preds2.to_csv('Fin4.csv', index=False)

In [ ]:
import lightgbm as lgb
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'learning_rate': 0.003,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 10,
    "num_leaves": 256,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}

gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='l1',
        early_stopping_rounds=1000)

In [ ]:
y_pred2 = gbm.predict(X_val, num_iteration=gbm.best_iteration_)
rms3 = np.sqrt(mean_squared_error(y_val, y_pred2))
print('all gives RMSE is:',rms3)

In [ ]:
Y_pred4=gbm.predict(Y,num_iteration=gbm.best_iteration_)
#label_encoder.fit(Y_pred4)
#pred_labels = label_encoder.inverse_transform(Y_pred)
pred_final5 = pd.DataFrame(Y_pred4)
#pred_final.insert(0, "Id", id_col, True)
pred_final5.describe(include='all')
pred_final5.to_csv('Final.csv', index=False)